In [1]:
import numpy as np
import matplotlib.pyplot as plt
import firebase_admin
from firebase_admin import credentials, firestore
from datetime import datetime

In [2]:
cred = credentials.Certificate("serviceAccountKey.json")
firebase_admin.initialize_app(cred)

In [10]:

db = firestore.client()
coleccion_tesla = db.collection("PreciosTesla")

datos_tesla = coleccion_tesla.order_by("precioCierre").get()
# Inicializa las listas de x (fechas) y y (precios de cierre)
x = []
y = []

# Llena las listas con los datos de Firebase
for dato in datos_tesla:
    fecha = dato.get('fecha')
    x.append(fecha)

    # Convierte el precio de cierre a un número (asumiendo que es un número)
    precio_cierre = float(dato.get('precioCierre'))
    y.append(precio_cierre)

# Convierte las listas a arrays de NumPy
x = np.array(x)
y = np.array(y)
print(x)
print(y)

[2010 2011 2012 2013 2016 2015 2017 2018 2019 2022 2020 2023 2021 2014]
[  1.78   1.9    2.26  10.03  14.25  16.    20.76  22.19  27.89 123.18
 235.22 236.08 352.26  14.83]


In [7]:
x, indices = np.unique(x, return_index=True)
y = y[indices]

In [8]:
def polynomial_evaluation(x, coefficients):
    """
    Evaluates a polynomial with given coefficients at a specific point x.

    Parameters:
    - x (float): The value at which the polynomial is evaluated.
    - coefficients (list): Coefficients of the polynomial in decreasing order.

    Returns:
    float: The result of the polynomial evaluation at x.
    """
    result = 0
    n = len(coefficients) - 1

    for coef in coefficients:
        result += coef * x**n
        n -= 1

    return result

x = np.array(x)
y = np.array(y)
anno = '31/11/2023'

for i in range(0, 10):
    coef = np.polyfit(x, y, i)
    p = np.polyval(coef, anno)

    print(f"Para grado {i} la predicción es {p}")

    x1 = np.linspace(1920, anno + 1, 1000)
    y1 = polynomial_evaluation(x1, coef)

    plt.figure(figsize=[20, 10])
    plt.title(f"Cantidad de litros vs año. Para grado: {i}")

    plt.scatter(x, y, s=120, c='blueviolet')
    plt.plot(x1, y1, "--", linewidth=3, color='orange')
    plt.scatter(anno, p, s=200, c='red')
    plt.yticks(range(100, 320, 20))
    plt.grid(True)
    ax = plt.gca()
    ax.set_xlabel("$años$")
    ax.set_ylabel("$Cantidad litros$")
    plt.show()

TypeError: unsupported operand type(s) for +: 'datetime.datetime' and 'float'

In [ ]:
anno = 2000
grado = np.arange(0, 30 + 1, 1)
aproxi = np.array([])

for i in grado:
    coef = np.polyfit(x, y, i)
    p = np.polyval(coef, anno)
    aproxi = np.append(aproxi, p)

    # Calcula el MSE de manera más eficiente sin bucle
    y_pred_vec = np.polyval(coef, x)
    MSE = np.mean((y - y_pred_vec)**2)
    print(f"Para grado {i} el MSE es: {MSE}")
plt.figure(figsize=[20, 10])
plt.title("Grado del polinomio vs predicción")
plt.plot(grado, aproxi, "--", linewidth=3, color='red')
plt.grid(True)
plt.show()